<h1 align=center > Clustering Cities In INDIA <h1>

### Introduction 
   India is one of the most populated country.There are 302 cities in India.

### Importing Required Libraries 

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install  folium 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Download and Explore Dataset

In [7]:
cities_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_in_India_by_population")[0]
cities_df.head()
    

,Rank,City,Population(2011)[3],Population(2001),State or union territory
0,1,Mumbai,12442373,11978450,Maharashtra
1,2,Delhi,11007835,9879172,Delhi
2,3,Bangalore,8436675,4301326,Karnataka
3,4,Hyderabad,6809970,3637483,Telangana
4,5,Ahmedabad,5570585,3520085,Gujarat


In [8]:
cities_df = cities_df.drop(['Rank','Population(2011)[3]','Population(2001)'],axis = 1)
cities_df = cities_df.rename(columns = {"Population(2011)[3]" : "p2011","Population(2001)" : "p2001"})
cities_df.head()

,City,State or union territory
0,Mumbai,Maharashtra
1,Delhi,Delhi
2,Bangalore,Karnataka
3,Hyderabad,Telangana
4,Ahmedabad,Gujarat


In [9]:
cities_df['City'] = cities_df['City'].apply(lambda x : x.split('[')[0])

In [10]:
longitude = []
latitude = []
for address in cities_df['City']:
    address = address+',India'
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(np.nan)
        longitude.append(np.nan)
    print(address, end=" ")
    
print('done')

Mumbai,India Delhi,India Bangalore,India Hyderabad,India Ahmedabad,India Chennai,India Kolkata,India Surat,India Pune,India Jaipur,India Kanpur,India Nagpur,India Lucknow,India Visakhapatnam,India Thane,India Bhopal,India Indore,India Pimpri-Chinchwad,India Patna,India Vadodara,India Ghaziabad,India Ludhiana,India Agra,India Nashik,India Faridabad,India Meerut,India Rajkot,India Kalyan-Dombivli,India Vasai-Virar,India Varanasi,India Srinagar,India Aurangabad,India Dhanbad,India Amritsar,India Navi Mumbai,India Allahabad,India Howrah,India Ranchi,India Gwalior,India Jabalpur,India Coimbatore,India Vijayawada,India Jodhpur,India Madurai,India Raipur,India Chandigarh,India Guwahati,India Solapur,India Hubli–Dharwad,India Mysore,India Tiruchirappalli,India Bareilly,India Aligarh,India Tiruppur,India Gurgaon,India Moradabad,India Jalandhar,India Bhubaneswar,India Salem,India Warangal,India Mira-Bhayandar,India Jalgaon,India Kota,India Guntur,India Bhiwandi,India Saharanpur,India Gorakhpur,I

In [11]:
cities_df['longitude'] = longitude
cities_df['latitude'] = latitude
cities_df.head()

,City,State or union territory,longitude,latitude
0,Mumbai,Maharashtra,72.835335,18.938771
1,Delhi,Delhi,77.221939,28.651718
2,Bangalore,Karnataka,77.591300,12.979120
3,Hyderabad,Telangana,78.461065,17.388786
4,Ahmedabad,Gujarat,72.579707,23.021624


In [14]:
cities_df.isnull().sum()

City                        0
State or union territory    0
longitude                   5
latitude                    5
dtype: int64

In [16]:
df = cities_df.dropna(axis = 0)
df.head()

,City,State or union territory,longitude,latitude
0,Mumbai,Maharashtra,72.835335,18.938771
1,Delhi,Delhi,77.221939,28.651718
2,Bangalore,Karnataka,77.591300,12.979120
3,Hyderabad,Telangana,78.461065,17.388786
4,Ahmedabad,Gujarat,72.579707,23.021624


In [17]:
df.isnull().sum()

City                        0
State or union territory    0
longitude                   0
latitude                    0
dtype: int64

In [18]:
address = 'India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of India are 22.3511148, 78.6677428.


In [22]:
map_india = folium.Map(location=[latitude,longitude],zoom_start = 5)

for lat, lng, city in zip(df['latitude'], df['longitude'],df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_india)  
    
map_india

### Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = '115HWRNCE1UQJRQZ1RRONOFWUWSG1SRN0UPETDWD3EMNHH2K' # your Foursquare ID
CLIENT_SECRET = 'ZLDIGK2HJD0K2T4NPMRMYZRT2P4Y1XMIYSZ0VR4MAHWA4F2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 115HWRNCE1UQJRQZ1RRONOFWUWSG1SRN0UPETDWD3EMNHH2K
CLIENT_SECRET:ZLDIGK2HJD0K2T4NPMRMYZRT2P4Y1XMIYSZ0VR4MAHWA4F2J


In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
#fuction to get venues in each city.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
city_venues = getNearbyVenues(names=df['City'],latitudes=df['latitude'],longitudes=df['longitude'])

Mumbai
Delhi
Bangalore
Hyderabad
Ahmedabad
Chennai
Kolkata
Surat
Pune
Jaipur
Kanpur
Nagpur
Lucknow
Visakhapatnam
Thane
Bhopal
Indore
Pimpri-Chinchwad
Patna
Vadodara
Ghaziabad
Ludhiana
Agra
Nashik
Faridabad
Meerut
Rajkot
Kalyan-Dombivli
Vasai-Virar
Varanasi
Srinagar
Aurangabad
Dhanbad
Amritsar
Navi Mumbai
Allahabad
Howrah
Ranchi
Gwalior
Jabalpur
Coimbatore
Vijayawada
Jodhpur
Madurai
Raipur
Chandigarh
Guwahati
Solapur
Hubli–Dharwad
Mysore
Tiruchirappalli
Bareilly
Aligarh
Tiruppur
Gurgaon
Moradabad
Jalandhar
Bhubaneswar
Salem
Warangal
Mira-Bhayandar
Jalgaon
Kota
Guntur
Bhiwandi
Saharanpur
Gorakhpur
Bikaner
Amravati
Noida
Jamshedpur
Bhilai
Cuttack
Firozabad
Kochi
Nellore
Bhavnagar
Dehradun
Durgapur
Asansol
Rourkela
Nanded
Kolhapur
Ajmer
Akola
Gulbarga
Jamnagar
Ujjain
Loni
Siliguri
Jhansi
Ulhasnagar
Jammu
Sangli-Miraj & Kupwad
Mangalore
Erode
Belgaum
Ambattur
Tirunelveli
Malegaon
Gaya
Thiruvananthapuram
Udaipur
Kakinada
Davanagere
Kozhikode
Maheshtala
Rajpur Sonarpur
Rajahmundry
Bokaro
Sout

In [38]:
city_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mumbai,18.938771,72.835335,Town House Cafe,18.938550,72.833464,Bar
1,Mumbai,18.938771,72.835335,Royal China,18.938715,72.832933,Chinese Restaurant
2,Mumbai,18.938771,72.835335,Sher-E-Punjab,18.937944,72.837853,Indian Restaurant
3,Mumbai,18.938771,72.835335,Cafe Excelsior,18.937701,72.833566,Café
4,Mumbai,18.938771,72.835335,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train Station


In [41]:
city_data = pd.get_dummies(city_venues[['Venue Category']],prefix="",prefix_sep="")
city_data['City'] = city_venues['City']
city_data.head()

,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Candy Store,Capitol Building,Castle,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Grocery Store,Gujarati Restaurant,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Nightclub,Optical Shop,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Snack Place,Spa,Sporting Goods Shop,Stadium,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [42]:
city_grouped = city_data.groupby('City').mean().reset_index()
city_grouped.head()

,City,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Candy Store,Capitol Building,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Grocery Store,Gujarati Restaurant,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Nightclub,Optical Shop,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Snack Place,Spa,Sporting Goods Shop,Stadium,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Wine Shop
0,Agartala,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,Agra,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.250000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Ahmedabad,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.200000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [43]:
city_grouped.shape

(151, 154)

## Cluster Cities

In [45]:
# set number of clusters
kclusters = 5

city_grouped_clustering = city_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 4, 4, 4, 4, 4, 4], dtype=int32)

In [53]:
l = list(kmeans.labels_)
ndf = pd.DataFrame()
ndf['Cluster label'] = l
ndf['City'] = city_grouped['City']
ndf.head()

,Cluster label,City
0,2,Agartala
1,2,Agra
2,2,Ahmedabad
3,1,Aizawl
4,4,Ajmer


In [54]:
final_df = pd.merge(ndf,df,on = 'City')
final_df.head()

,Cluster label,City,State or union territory,longitude,latitude
0,2,Agartala,Tripura,91.282382,23.831238
1,2,Agra,Uttar Pradesh,78.009816,27.175255
2,2,Ahmedabad,Gujarat,72.579707,23.021624
3,1,Aizawl,Mizoram,92.720930,23.741409
4,4,Ajmer,Rajasthan,74.639000,26.469100


In [55]:
fianl_df.shape

(152, 5)

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(fianl_df['latitude'],final_df['longitude'],final_df['City'],final_df['Cluster label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters